In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Mariem\\Desktop\\Learning_Journey\\End_To_End_Chest_Cancer_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Mariem\\Desktop\\Learning_Journey\\End_To_End_Chest_Cancer_Detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Mariem17-17/End_To_End_Chest_Cancer_Detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Mariem17-17"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b2cb51c880480b3e7ecabc262adbc2bd575b7555"

In [6]:
import tensorflow as tf

e:\anaconda3\envs\test\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from CNN_classifier.constants import *
from CNN_classifier.utils.commons import read_yaml, create_directories, save_json

In [10]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/Mariem17-17/End_To_End_Chest_Cancer_Detection.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 102 images belonging to 2 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 693ms/step - accuracy: 0.8235 - loss: 0.2642


2026/01/08 12:36:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/08 12:36:06 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2026/01/08 12:55:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run placid-worm-227 at: https://dagshub.com/Mariem17-17/End_To_End_Chest_Cancer_Detection.mlflow/#/experiments/0/runs/e271ce6f7e3a4d748448697bd3cdef43
🧪 View experiment at: https://dagshub.com/Mariem17-17/End_To_End_Chest_Cancer_Detection.mlflow/#/experiments/0
